In [21]:
import os
from os import path
import h5py
import time
import sys

sys.path.append('../../../rai-fork/rai/ry')
import libry as ry
import numpy as np
import matplotlib.pyplot as plt

from src.simulation_utils import *

In [22]:
load_dir = '../dataGeneration_vF/data/object'
ind = 2

filename = [fn for fn in os.listdir(load_dir) if fn.endswith('.hdf5')][ind]
    
data_obj = h5py.File(path.join(load_dir, filename), mode='r')
filename = data_obj['filename'][()].decode()
mesh_coll_name = path.join('data/meshes_coll', filename)
size = data_obj['size'][()]
print(size)
mass = data_obj['mass'][()]
com = data_obj['com'][:]
data_obj.close()

C = ry.Config()
C.addFile('world4.g')
mug = C.addMeshFrame(mesh_coll_name, 'mug', mass=mass, com=com)
mug.setPosition([0.,-0.2,1.0]).setQuaternion([1,0,0,1.])
C.selectJoints([j for j in C.getJointNames() if j not in ['L_finger']])

S = C.simulation(ry.SimulatorEngine.bullet, 1)
for _ in range(500):
    S.step([], 0.01, ry.ControlMode.none)
fInit = C.getFrameState()
qInit = C.getJointState()

0.10730413538236316


In [23]:
C.selectJoints([j for j in C.getJointNames() if j not in ['L_finger']])
V = ry.ConfigurationViewer()
V.setConfiguration(C)

C.addFrame('graspedObj', 'gripperCenter').setRelativePosition([0,0,-.1])

stepsPerPhase = 10
komo = C.komo(3., stepsPerPhase, 5., 2, False)
komo.verbose(3)
# komo.animateOptimization(True)

Sk = [[1., 3.], ry.SY.stable, ["gripper", "mug"]]
komo.addSkeleton(Sk)

komo.add_qControlObjective([], 2)
komo.add_qControlObjective([], 1)

colls = ["gripper_coll", 
         "L_finger_coll1", "L_finger_coll2",
         "R_finger_coll1", "R_finger_coll2"]


komo.addObjective([1.], ry.FS.positionRel, ["gripperCenter", "mug"], ry.OT.eq, [1e0], target=[0.1,0,0])
komo.addObjective([1.], ry.FS.vectorZ, ["gripperCenter"], ry.OT.eq, [1e0], target=[0,1/np.sqrt(2),1/np.sqrt(2)])
komo.addObjective([1.], ry.FS.scalarProductXZ, ["gripperCenter", "world"], ry.OT.eq, [1e0])
komo.addObjective([.7, 1.], ry.FS.quaternion, ["gripperCenter"], ry.OT.eq, [1e0], order=1)
komo.addObjective([.7, 1.], ry.FS.positionRel, ["mug", "gripperCenter"], ry.OT.eq, [1e1], target=[0,0,-.1], order=2)
komo.addObjective([1.], ry.FS.qItself, C.getJointNames(), ry.OT.eq, [1e1], order=1)



komo.addObjective([1., 2.], ry.FS.vectorZ, ["mug"], ry.OT.eq, [1e0], order=1)
komo.addObjective([2., 3.], ry.FS.position, ["gripperCenter"], ry.OT.eq, [1e0], target=[.15, .3, 1.3])

komo.addObjective([3.], ry.FS.vectorZ, ["mug"], ry.OT.eq, [1e0], target=[1/np.sqrt(2), 0, -1/np.sqrt(2)])
komo.addObjective([3.], ry.FS.qItself, C.getJointNames(), ry.OT.eq, [1e1], order=1)

komo.optimize(0.1)

V=komo.view()

In [25]:
def stepTo(S, q, tau=0.1):
    N = int(tau/0.01)
    dq = q - S.get_q()
    for i in range(N):
        S.step(S.get_q()+dq/N, tau/N, ry.ControlMode.position)
        time.sleep(tau/N)
        
RealWorld = ry.Config()
RealWorld.addFile('world4.g')
mug = RealWorld.addMeshFrame(mesh_coll_name, 'mug', mass=mass, com=com)
RealWorld.setFrameState(fInit)
for i in range(10):
    pos = mug.getPosition() + np.array([0,0,.05+.05*i])
    b = RealWorld.addFrame('ball_'+str(i)).setShape(ry.ST.sphere, [0.015]).setColor([.8,.6,.6])
    b.setPosition(pos).setMass(0.00001)#.addAttribute('friction',0.0)
RealWorld.selectJoints([j for j in RealWorld.getJointNames() if j not in ['L_finger']])
camera_name_list = ['camera_'+str(i) for i in range(4)]
camera = RealWorld.cameraView()
for camera_name in camera_name_list: 
    camera.addSensorFromFrame(camera_name)

t = 0
save_dir = 'screenshots/imitation_ref/'

S = RealWorld.simulation(ry.SimulatorEngine.bullet, 4)
tau = 0.1
for _ in range(500):
    S.step([], 0.01, ry.ControlMode.none)
    
for t in range(stepsPerPhase):
    C.setFrameState(komo.getConfiguration(t))
    q = C.getJointState()
    stepTo(S, q)
    plt.imsave(save_dir+str(t).zfill(3)+'.png', S.getScreenshot()[::-1])
    t += 1

    
S.closeGripper("gripper", speed=3., objFrameName="mug")
while not (S.getGripperIsGrasping("gripper") or S.getGripperIsClose("gripper")):
    stepTo(S, S.get_q())
    
plt.imsave(save_dir+str(t).zfill(3)+'.png', S.getScreenshot()[::-1])
t += 1
for t in range(stepsPerPhase,2*stepsPerPhase):
    C.setFrameState(komo.getConfiguration(t))
    q = C.getJointState()
    stepTo(S, q, 0.2)
    plt.imsave(save_dir+str(t).zfill(3)+'.png', S.getScreenshot()[::-1])
    t += 1
    
out1 = get_all_images(RealWorld, 
                     camera, 
                     camera_name_list, 
                     ['mug'], 
                     r=0.15, )
    
    
for t in range(2*stepsPerPhase, 3*stepsPerPhase):
    C.setFrameState(komo.getConfiguration(t))
    q = C.getJointState()
    stepTo(S, q, 0.2)
    plt.imsave(save_dir+str(t).zfill(3)+'.png', S.getScreenshot()[::-1])
    t += 1

out2 = get_all_images(RealWorld, 
                     camera, 
                     camera_name_list, 
                     ['mug'], 
                     r=0.15, )

for i in range(10):
    for _ in range(10): S.step([], 0.01, ry.ControlMode.none)
    plt.imsave(save_dir+str(t).zfill(3)+'.png', S.getScreenshot()[::-1])
    t += 1

In [16]:
with h5py.File('target1.hdf5', mode='w') as f:
    f.create_dataset('rgb', data=out1[2])
    f.create_dataset('projection', data=out1[3])
    f.create_dataset('obj_pos', data=out1[4])
    f.create_dataset('obj_r', data=out1[5])
    
with h5py.File('target2.hdf5', mode='w') as f:
    f.create_dataset('rgb', data=out2[2])
    f.create_dataset('projection', data=out2[3])
    f.create_dataset('obj_pos', data=out2[4])
    f.create_dataset('obj_r', data=out2[5])

In [26]:
os.system('ffmpeg -r 5 -pattern_type glob -i \'screenshots/imitation_ref/*.png\' -c:v libx264 imitation_ref.mp4')

0